# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [3]:
#Search for the TLDR Number 006179570C, and scrape the information on that company.
button = driver.find_element_by_xpath('//*[@id="mcrbutton"]')
driver.execute_script('arguments[0].scrollIntoView(true)', button)
button.click()

In [4]:
license_data=driver.find_element_by_xpath('//*[@id="mcrdata"]').send_keys('006179570C')

In [5]:
button = driver.find_element_by_xpath('//*[@id="submit3"]')
driver.execute_script('arguments[0].scrollIntoView(true)', button)
button.click()

In [6]:
system =driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]')
for row in system:
    cells = row.find_elements_by_tag_name('td')
    name=cells[2].text
    operator=cells[4].text
    phone=cells[6].text

In [7]:
#more information 
more_system=driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]')
for row in more_system:
    cells=row.find_elements_by_tag_name('td')
    status=cells[1].text
    physical_adress=cells[3].text
    address=physical_adress.split("\n")[-2:]

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [8]:
# I did not know that in the previous step we couldnt print all the detaisl. I changed it like a thousand times because it did 
#not give me an output...silly me!!!!!!
def get_license_information (row):
    button = driver.find_element_by_xpath('//*[@id="mcrbutton"]')
    driver.execute_script('arguments[0].scrollIntoView(true)', button)
    button.click()
    
    license_data=driver.find_element_by_xpath('//*[@id="mcrdata"]').send_keys(row['TDLR_Number'])
    
    button = driver.find_element_by_xpath('//*[@id="submit3"]')
    driver.execute_script('arguments[0].scrollIntoView(true)', button)
    button.click()
    
    system =driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]')
    for row in system:
        cells = row.find_elements_by_tag_name('td')
        name=cells[2].text
        operator=cells[4].text
        phone=cells[6].text
    
    more_system=driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]')
    for row in more_system:
        cells=row.find_elements_by_tag_name('td')
        status=cells[1].text
        physical_adress=cells[3].text
        address=physical_adress.split("\n")[-2:]
    

    return pd.Series({
        'name': name,
        'operator': operator,
        'status': status,
        'address': address,
        'phone': phone    
    })

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [9]:
import pandas as pd
df=pd.read_csv('trucks-subset.csv')
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


In [10]:
new_df=df

In [ ]:
truck_companies = new_df.apply(get_license_information, axis=1).join(new_df)
truck_companies

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [56]:
pd.set_option('display.max_colwidth', -1)

In [57]:
def display_url(row):
    url='https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+(row['TDLR_Number'])
    return pd.Series({
        'url': url})

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [ ]:
urls= df.apply(display_url, axis=1).join(df)
urls

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [59]:
def all_truckcompanies(row): 
    
    driver.get(row['url'])
    
    system=driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]')
    for row in system:
        cells=row.find_elements_by_tag_name('td')
        name=cells[2].text
        owner=cells[4].text
        phone=cells[6].text

    more_system=driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]')
    for row in more_system:
        cells=row.find_elements_by_tag_name('td')
        status=cells[1].text
        physical=cells[3].text
        address=physical.split("\n")[-2:]
    
    return name, owner, phone, status, address

In [ ]:
urls.apply(all_truckcompanies, axis=1)

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [63]:
def all_truckcompanies(row): 
    
    driver.get(row['url'])
    
    system=driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]')
    for row in system:
        cells=row.find_elements_by_tag_name('td')
        name=cells[2].text
        owner=cells[4].text
        phone=cells[6].text

    more_system=driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]')
    for row in more_system:
        cells=row.find_elements_by_tag_name('td')
        status=cells[1].text
        physical=cells[3].text
        address=physical.split("\n")[-2:]
    
    return name, owner, phone, status, address

In [ ]:
complete_companies= urls.apply(all_truckcompanies, axis=1)
complete_companies

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [ ]:
complete_companies.to_csv('tow_trucks_extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [ ]:
dataframe=pd.read_csv('tow_trucks_extended.csv')
dataframe

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [68]:
tow_trucks=pd.read_csv('tow-trucks.csv')
tow_trucks.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF


In [ ]:
def displaying_url(row):
    url='https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+(row['TDLR_Number'])
    return pd.Series({
        'url': url})
all_tow_trucks= tow_trucks.apply(displaying_url, axis=1).join(tow_trucks)
all_tow_trucks.head()

In [ ]:
final= all_tow_trucks.apply(all_truckcompanies, axis=1).join(all_tow_trucks)